  <picture>
  <source srcset="https://raw.githubusercontent.com/Archfx/RACKETutes/main/images/racketutes-w.svg" media="(prefers-color-scheme: dark)">
    <img src="https://raw.githubusercontent.com/Archfx/RACKETutes/main/images/racketutes.svg">
  </picture>

Veri(fy)log
===

In previous posts, we discussed Racket and Rosette. This is the post where we are going to verify our hardware designs using the infrastructure we have developed so far. We will be looking at verifying Verilog hardware designs with Rosette. Later in this tutorial, we will be validating a simple design based on Verilog implemented picoRV processor that works with a simple firmware that we implemented in one of my previous blog posts.

We need to follow several steps to convert Verilog designs to Racket (.rkt) native formats. For this purpose, we need [Yosys]() and [rtlv]().



### Verilog to SMT (.v -> .smt2)

First, we need to convert the Verilog implementation into SMT space, so that our underlying Z3 solver can understand the implementation. For this purpose we use [Yosys](). Let's look at a simple Verilog implementation of a 32-bit adder.

```verilog
module adder (
    input [31:0] a,
    input [31:0] b,
    output [31:0] c
);
assign c = a + b;
endmodule
```

In order to convert this into smt2 format we can use the following command

```shell
yosys -p 'read_verilog -defer adder.v' -p 'prep -flatten -top adder -nordff' -p 'write_smt2 -stdt adder.smt2'
```

You can generate the command similar commands for Yosys from [here](https://archfx.github.io/cad.html). 

### SMT to rkt (.smt2 -> .rkt)
This will generate the smt2 transformed verilog implementation. We need to add the language definition from the `rtlv` to identify it via Rosette. For that, we can use the following command.

```shell
echo #lang yosys $ cat adder.smt > adder.rkt
```

This will generate the `adder.rkt` file which we will be used to refer to the implementation.


In [1]:
#lang iracket/lang #:require rosette/safe 

In [4]:
(require rackunit
         "racketutes/verilog/test/verilog/adder.rkt"
         rosutil
         yosys
         (only-in racket/base string-append parameterize regexp-match))

Specification

In [5]:
;spec
(define (abs-spec x y)
   display (bvadd x y))

In [6]:
(test-case "eq1"
  (define s0 (update-adder_s
              (new-symbolic-adder_s)
              [a (bv 4 32)]
              [b (bv 5 32)]))
    (assert (check-equal? (|adder_n c| s0) (abs-spec (bv 4 32) (bv 5 32)))))